# Transcription Plugin Interface

> Domain-specific plugin interface for audio transcription plugins

In [ ]:
#| default_exp plugin_interface

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from abc import ABC, abstractmethod
import logging
from typing import Optional, Dict, Any, Union, List, Tuple, Generator
from pathlib import Path
import json

# Import generic plugin infrastructure from cjm-plugin-system
from cjm_plugin_system.core.interface import PluginInterface
from cjm_plugin_system.core.metadata import PluginMeta

# Import domain-specific types
from cjm_transcription_plugin_system.core import AudioData, TranscriptionResult

In [ ]:
#| export
class TranscriptionPlugin(PluginInterface):
    """Transcription-specific plugin interface.
    
    This extends the generic PluginInterface with transcription-specific
    requirements like supported audio formats and the execute signature.
    
    All transcription plugins must implement this interface.
    """

    entry_point_group = "transcription.plugins"
    
    @property
    @abstractmethod
    def supported_formats(
        self
    ) -> List[str]:  # List of file extensions this plugin can process
        """List of supported audio formats (e.g., ['wav', 'mp3']).
        
        Returns:
            List of file extensions without the dot (e.g., ['wav', 'mp3', 'flac'])
        """
        pass
    
    @abstractmethod
    def execute(
        self,
        audio: Union[AudioData, str, Path],  # Audio data or path to audio file
        **kwargs  # Additional plugin-specific parameters
    ) -> TranscriptionResult:  # Transcription result with text and metadata
        """Transcribe audio to text.
        
        Args:
            audio: Audio data (AudioData object), file path (str), or Path object
            **kwargs: Additional plugin-specific parameters (e.g., language, model)
            
        Returns:
            TranscriptionResult containing transcribed text, confidence, segments, and metadata
        """
        pass

## TranscriptionPlugin Interface

The `TranscriptionPlugin` interface extends the generic `PluginInterface` from `cjm-plugin-system` with transcription-specific requirements:

- **`supported_formats`** property: List of audio file formats this plugin can handle
- **`execute`** method signature: Takes audio input and returns `TranscriptionResult`

All generic plugin functionality (configuration management, validation, streaming support, etc.) is inherited from the base `PluginInterface` class.

In [ ]:
# Test PluginMeta dataclass
meta = PluginMeta(
    name="test_plugin",
    version="1.0.0",
    description="A test plugin",
    author="Test Author"
)

print("PluginMeta instance:")
print(meta)
print(f"\nName: {meta.name}")
print(f"Version: {meta.version}")
print(f"Enabled: {meta.enabled}")
print(f"Instance: {meta.instance}")

# Test with minimal arguments
minimal_meta = PluginMeta(name="minimal", version="0.1.0")
print(f"\nMinimal PluginMeta: {minimal_meta}")

# Test equality
meta_copy = PluginMeta(name="minimal", version="0.1.0")
print(f"Equality test: {minimal_meta == meta_copy}")

PluginMeta instance:
PluginMeta(name='test_plugin', version='1.0.0', description='A test plugin', author='Test Author', package_name='', instance=None, enabled=True)

Name: test_plugin
Version: 1.0.0
Enabled: True
Instance: None

Minimal PluginMeta: PluginMeta(name='minimal', version='0.1.0', description='', author='', package_name='', instance=None, enabled=True)
Equality test: True


## Testing ExamplePlugin

Let's test a simple transcription plugin implementation.

In [ ]:
class ExamplePlugin(TranscriptionPlugin):
    """An example transcription plugin implementation with configuration schema."""

    def __init__(self):
        self.logger = logging.getLogger(f"{__name__}.{type(self).__name__}")
        self.config = {}
        self.model = None
    
    @property
    def name(self) -> str:
        return "example_plugin"
    
    @property
    def version(self) -> str:
        return "1.0.0"

    @property
    def supported_formats(self) -> List[str]:
        return ["wav", "mp3", "flac"]

    @staticmethod
    def get_config_schema() -> Dict[str, Any]:
        """Return the configuration schema for this plugin."""
        return {
            "type": "object",
            "properties": {
                "model": {
                    "type": "string",
                    "enum": ["tiny", "base", "small", "medium", "large"],
                    "default": "base",
                    "description": "Model size to use for transcription"
                },
                "language": {
                    "type": "string",
                    "default": "auto",
                    "description": "Language code (e.g., 'en', 'es') or 'auto' for detection"
                },
                "batch_size": {
                    "type": "integer",
                    "minimum": 1,
                    "maximum": 32,
                    "default": 8,
                    "description": "Batch size for processing"
                },
                "enable_vad": {
                    "type": "boolean",
                    "default": True,
                    "description": "Enable voice activity detection"
                }
            },
            "required": ["model"]
        }
    
    def get_current_config(self) -> Dict[str, Any]:
        """Return the current configuration."""
        # Merge defaults with actual config
        defaults = self.get_config_defaults()
        return {**defaults, **self.config}
    
    def initialize(self, config: Optional[Dict[str, Any]] = None) -> None:
        """Initialize the plugin."""
        if config:
            is_valid, error = self.validate_config(config)
            if not is_valid:
                raise ValueError(f"Invalid configuration: {error}")
        
        # Merge provided config with defaults
        defaults = self.get_config_defaults()
        self.config = {**defaults, **(config or {})}
        
        self.logger.info(f"Initializing {self.name} with config: {self.config}")
        
        # Simulate loading a model based on config
        model_name = self.config.get("model", "base")
        self.model = f"MockModel-{model_name}"
    
    def execute(
        self,
        audio: Union[AudioData, str, Path],
        **kwargs
    ) -> TranscriptionResult:
        """Execute the plugin's functionality."""
        self.logger.info(f"Example plugin executed with model: {self.model}")
        self.logger.info(f"Config: {self.config}")
        
        # Mock transcription result
        return TranscriptionResult(
            text=f"Transcription using {self.model}",
            confidence=0.95,
            segments=[],
            metadata={"model": self.config.get("model")}
        )

    def is_available(self) -> bool:
        """Check availability."""
        return True
    
    def cleanup(self) -> None:
        """Clean up resources."""
        self.logger.info(f"Cleaning up {self.name}")
        self.model = None

In [ ]:
#| eval: false
logging.basicConfig(level=logging.INFO)

example_plugin = ExamplePlugin()
example_plugin.initialize()
transcription_result = example_plugin.execute("test_audio.mp3")
example_plugin.cleanup()
print(f"Result text: {transcription_result.text}")
print(f"Result confidence: {transcription_result.confidence}")

INFO:__main__.ExamplePlugin:Initializing example_plugin with config: {'model': 'base', 'language': 'auto', 'batch_size': 8, 'enable_vad': True}
INFO:__main__.ExamplePlugin:Example plugin executed with model: MockModel-base
INFO:__main__.ExamplePlugin:Config: {'model': 'base', 'language': 'auto', 'batch_size': 8, 'enable_vad': True}
INFO:__main__.ExamplePlugin:Cleaning up example_plugin


Result text: Transcription using MockModel-base
Result confidence: 0.95


In [ ]:
# Test the configuration schema functionality
plugin = ExamplePlugin()

# Get the configuration schema
schema = plugin.get_config_schema()
print("Configuration Schema:")
print(json.dumps(schema, indent=2))

print("\n" + "="*50 + "\n")

# Get default configuration
defaults = plugin.get_config_defaults()
print("Default Configuration:")
print(json.dumps(defaults, indent=2))

print("\n" + "="*50 + "\n")

# Initialize with partial config (using defaults for missing values)
plugin.initialize({"model": "small", "language": "en"})
print("Current Configuration after initialization:")
print(json.dumps(plugin.get_current_config(), indent=2))

print("\n" + "="*50 + "\n")

# Test configuration validation
test_configs = [
    ({"model": "tiny"}, "Valid config with minimal settings"),
    ({"model": "invalid_model"}, "Invalid model name"),
    ({"batch_size": 100}, "Missing required 'model' field"),
    ({"model": "base", "batch_size": 100}, "Batch size exceeds maximum"),
    ({"model": "base", "batch_size": "not_a_number"}, "Invalid type for batch_size"),
]

for config, description in test_configs:
    is_valid, error = plugin.validate_config(config)
    print(f"{description}:")
    print(f"  Config: {config}")
    print(f"  Valid: {is_valid}")
    if error:
        print(f"  Error: {error[:80]}...")  # Truncate long errors
    print()

INFO:__main__.ExamplePlugin:Initializing example_plugin with config: {'model': 'small', 'language': 'en', 'batch_size': 8, 'enable_vad': True}


Configuration Schema:
{
  "type": "object",
  "properties": {
    "model": {
      "type": "string",
      "enum": [
        "tiny",
        "base",
        "small",
        "medium",
        "large"
      ],
      "default": "base",
      "description": "Model size to use for transcription"
    },
    "language": {
      "type": "string",
      "default": "auto",
      "description": "Language code (e.g., 'en', 'es') or 'auto' for detection"
    },
    "batch_size": {
      "type": "integer",
      "minimum": 1,
      "maximum": 32,
      "default": 8,
      "description": "Batch size for processing"
    },
    "enable_vad": {
      "type": "boolean",
      "default": true,
      "description": "Enable voice activity detection"
    }
  },
  "required": [
    "model"
  ]
}


Default Configuration:
{
  "model": "base",
  "language": "auto",
  "batch_size": 8,
  "enable_vad": true
}


Current Configuration after initialization:
{
  "model": "small",
  "language": "en",
  "batch_size": 8,

## Example: Whisper Plugin Implementation

In [ ]:
class WhisperPlugin(TranscriptionPlugin):
    """Example Whisper transcription plugin with comprehensive configuration."""
    
    def __init__(self):
        self.logger = logging.getLogger(f"{__name__}.{type(self).__name__}")
        self.config = {}
        self.model = None
        self.processor = None
    
    @property
    def name(self) -> str:
        return "whisper"
    
    @property
    def version(self) -> str:
        return "1.0.0"
    
    @property
    def supported_formats(self) -> List[str]:
        return ["wav", "mp3", "flac", "m4a", "ogg", "webm"]

    @staticmethod
    def get_config_schema() -> Dict[str, Any]:
        """Return comprehensive Whisper configuration schema."""
        return {
            "$schema": "http://json-schema.org/draft-07/schema#",
            "type": "object",
            "title": "Whisper Configuration",
            "properties": {
                "model": {
                    "type": "string",
                    "enum": ["tiny", "tiny.en", "base", "base.en", "small", "small.en", 
                            "medium", "medium.en", "large", "large-v1", "large-v2", "large-v3"],
                    "default": "base",
                    "description": "Whisper model size. Larger models are more accurate but slower."
                },
                "device": {
                    "type": "string",
                    "enum": ["cpu", "cuda", "mps", "auto"],
                    "default": "auto",
                    "description": "Computation device for inference"
                },
                "compute_type": {
                    "type": "string",
                    "enum": ["default", "float16", "float32", "int8", "int8_float16"],
                    "default": "default",
                    "description": "Model precision/quantization"
                },
                "language": {
                    "type": ["string", "null"],
                    "default": None,
                    "description": "Language code (e.g., 'en', 'es', 'fr') or null for auto-detection",
                    "examples": ["en", "es", "fr", "de", "ja", "zh", None]
                },
                "task": {
                    "type": "string",
                    "enum": ["transcribe", "translate"],
                    "default": "transcribe",
                    "description": "Task to perform (transcribe keeps original language, translate converts to English)"
                },
                "temperature": {
                    "type": "number",
                    "minimum": 0.0,
                    "maximum": 1.0,
                    "default": 0.0,
                    "description": "Sampling temperature. 0 for deterministic, higher values for more variation"
                },
                "beam_size": {
                    "type": "integer",
                    "minimum": 1,
                    "maximum": 10,
                    "default": 5,
                    "description": "Beam search width. Higher values may improve accuracy but are slower"
                },
                "best_of": {
                    "type": "integer",
                    "minimum": 1,
                    "maximum": 10,
                    "default": 5,
                    "description": "Number of candidates when sampling with non-zero temperature"
                },
                "patience": {
                    "type": "number",
                    "minimum": 0.0,
                    "maximum": 2.0,
                    "default": 1.0,
                    "description": "Beam search patience factor"
                },
                "length_penalty": {
                    "type": ["number", "null"],
                    "default": None,
                    "description": "Exponential length penalty during beam search"
                },
                "suppress_tokens": {
                    "type": ["array", "string"],
                    "items": {"type": "integer"},
                    "default": "-1",
                    "description": "Token IDs to suppress. '-1' for default suppression, empty array for none"
                },
                "initial_prompt": {
                    "type": ["string", "null"],
                    "default": None,
                    "description": "Optional text to provide as prompt for first window"
                },
                "condition_on_previous_text": {
                    "type": "boolean",
                    "default": True,
                    "description": "Use previous output as prompt for next window"
                },
                "no_speech_threshold": {
                    "type": "number",
                    "minimum": 0.0,
                    "maximum": 1.0,
                    "default": 0.6,
                    "description": "Threshold for detecting silence"
                },
                "compression_ratio_threshold": {
                    "type": "number",
                    "minimum": 1.0,
                    "maximum": 10.0,
                    "default": 2.4,
                    "description": "Threshold for detecting repetition"
                },
                "logprob_threshold": {
                    "type": "number",
                    "default": -1.0,
                    "description": "Average log probability threshold"
                },
                "word_timestamps": {
                    "type": "boolean",
                    "default": False,
                    "description": "Extract word-level timestamps"
                },
                "prepend_punctuations": {
                    "type": "string",
                    "default": "\"'“¿([{-",
                    "description": "Punctuations to merge with next word"
                },
                "append_punctuations": {
                    "type": "string",
                    "default": "\"'.。,，!！?？:：”)]}、",
                    "description": "Punctuations to merge with previous word"
                },
                "vad_filter": {
                    "type": "boolean",
                    "default": False,
                    "description": "Enable voice activity detection filter"
                },
                "vad_parameters": {
                    "type": "object",
                    "properties": {
                        "threshold": {
                            "type": "number",
                            "minimum": 0.0,
                            "maximum": 1.0,
                            "default": 0.5
                        },
                        "min_speech_duration_ms": {
                            "type": "integer",
                            "minimum": 0,
                            "default": 250
                        },
                        "max_speech_duration_s": {
                            "type": "number",
                            "minimum": 0,
                            "default": 3600
                        }
                    },
                    "default": {}
                }
            },
            "required": ["model"],
            "additionalProperties": False
        }
    
    def get_current_config(self) -> Dict[str, Any]:
        """Return current configuration with all defaults applied."""
        defaults = self.get_config_defaults()
        current = {**defaults, **self.config}
        
        # Handle nested vad_parameters
        if "vad_parameters" in current and isinstance(current["vad_parameters"], dict):
            vad_defaults = {
                "threshold": 0.5,
                "min_speech_duration_ms": 250,
                "max_speech_duration_s": 3600
            }
            current["vad_parameters"] = {**vad_defaults, **current["vad_parameters"]}
        
        return current
    
    def initialize(self, config: Optional[Dict[str, Any]] = None) -> None:
        """Initialize the Whisper model with configuration."""
        if config:
            is_valid, error = self.validate_config(config)
            if not is_valid:
                raise ValueError(f"Invalid configuration: {error}")
        
        # Merge with defaults
        defaults = self.get_config_defaults()
        self.config = {**defaults, **(config or {})}
        
        self.logger.info(f"Initializing Whisper with config: {self.config}")
        
        # In a real implementation, this would load the actual Whisper model
        # For example:
        # import whisper
        # self.model = whisper.load_model(self.config["model"], device=self.config["device"])
        
        # Mock implementation
        self.model = f"WhisperModel-{self.config['model']}"
        self.processor = f"WhisperProcessor-{self.config['device']}"
    
    def execute(
        self,
        audio: Union[AudioData, str, Path],
        **kwargs
    ) -> TranscriptionResult:
        """Transcribe audio using Whisper."""
        if not self.model:
            raise RuntimeError("Plugin not initialized. Call initialize() first.")
        
        # Override config with any provided kwargs
        exec_config = {**self.config, **kwargs}
        
        self.logger.info(f"Transcribing with Whisper model: {self.model}")
        self.logger.info(f"Execution config: {exec_config}")
        
        # In a real implementation, this would:
        # 1. Load/preprocess audio
        # 2. Run Whisper inference
        # 3. Post-process results
        
        # Mock transcription result
        return TranscriptionResult(
            text=f"Mock transcription using {exec_config['model']} model",
            confidence=0.95,
            segments=[
                {
                    "start": 0.0,
                    "end": 2.5,
                    "text": "Mock transcription",
                    "confidence": 0.96
                },
                {
                    "start": 2.5,
                    "end": 5.0,
                    "text": f"using {exec_config['model']} model",
                    "confidence": 0.94
                }
            ],
            metadata={
                "model": exec_config["model"],
                "language": exec_config.get("language", "auto-detected"),
                "device": exec_config["device"],
                "task": exec_config["task"]
            }
        )
    
    def is_available(self) -> bool:
        """Check if Whisper dependencies are available."""
        try:
            # In real implementation, check for whisper package
            # import whisper
            # return True
            return True  # Mock always available
        except ImportError:
            return False
    
    def cleanup(self) -> None:
        """Clean up model from memory."""
        self.logger.info("Cleaning up Whisper model")
        self.model = None
        self.processor = None
        # In real implementation: del self.model, torch.cuda.empty_cache(), etc.

In [ ]:
# Test the Whisper plugin
whisper_plugin = WhisperPlugin()

# Get a subset of the schema for display (it's quite large)
schema = whisper_plugin.get_config_schema()
print("Whisper Configuration Schema (subset):")
print("Available models:", schema["properties"]["model"]["enum"])
print("Available devices:", schema["properties"]["device"]["enum"])
print("\nRequired fields:", schema.get("required", []))

print("\n" + "="*50 + "\n")

# Test initialization with different configurations
configs_to_test = [
    {"model": "tiny"},
    {"model": "large-v3", "device": "cuda", "language": "en"},
    {"model": "base", "temperature": 0.2, "beam_size": 3, "word_timestamps": True}
]

for config in configs_to_test:
    print(f"Initializing with config: {config}")
    whisper_plugin.initialize(config)
    current = whisper_plugin.get_current_config()
    print(f"Current model: {current['model']}")
    print(f"Current device: {current['device']}")
    print(f"Word timestamps: {current['word_timestamps']}")
    
    # Execute transcription
    result = whisper_plugin.execute("dummy_audio.wav")
    print(f"Result: {result.text}")
    print("-" * 30)

INFO:__main__.WhisperPlugin:Initializing Whisper with config: {'model': 'tiny', 'device': 'auto', 'compute_type': 'default', 'language': None, 'task': 'transcribe', 'temperature': 0.0, 'beam_size': 5, 'best_of': 5, 'patience': 1.0, 'length_penalty': None, 'suppress_tokens': '-1', 'initial_prompt': None, 'condition_on_previous_text': True, 'no_speech_threshold': 0.6, 'compression_ratio_threshold': 2.4, 'logprob_threshold': -1.0, 'word_timestamps': False, 'prepend_punctuations': '"\'“¿([{-', 'append_punctuations': '"\'.。,，!！?？:：”)]}、', 'vad_filter': False, 'vad_parameters': {}}
INFO:__main__.WhisperPlugin:Transcribing with Whisper model: WhisperModel-tiny
INFO:__main__.WhisperPlugin:Execution config: {'model': 'tiny', 'device': 'auto', 'compute_type': 'default', 'language': None, 'task': 'transcribe', 'temperature': 0.0, 'beam_size': 5, 'best_of': 5, 'patience': 1.0, 'length_penalty': None, 'suppress_tokens': '-1', 'initial_prompt': None, 'condition_on_previous_text': True, 'no_speech_th

Whisper Configuration Schema (subset):
Available models: ['tiny', 'tiny.en', 'base', 'base.en', 'small', 'small.en', 'medium', 'medium.en', 'large', 'large-v1', 'large-v2', 'large-v3']
Available devices: ['cpu', 'cuda', 'mps', 'auto']

Required fields: ['model']


Initializing with config: {'model': 'tiny'}
Current model: tiny
Current device: auto
Word timestamps: False
Result: Mock transcription using tiny model
------------------------------
Initializing with config: {'model': 'large-v3', 'device': 'cuda', 'language': 'en'}
Current model: large-v3
Current device: cuda
Word timestamps: False
Result: Mock transcription using large-v3 model
------------------------------
Initializing with config: {'model': 'base', 'temperature': 0.2, 'beam_size': 3, 'word_timestamps': True}
Current model: base
Current device: auto
Word timestamps: True
Result: Mock transcription using base model
------------------------------


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()